<a href="https://colab.research.google.com/github/yeyevtushenko/AI/blob/Lesson13.03.2024/Lesson13_03_2024_AI_P_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [28]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [29]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів(лише `price`), розділіть на тренувальну та тестову чатини

In [30]:
df.shape

(1553, 4)

In [31]:
def remove_outliers(df, column_name):
    column = df[column_name]

    q1 = column.quantile(q=0.25)
    q3 = column.quantile(q=0.75)

    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    mask = (column >= lower) & (column <= upper)

    return df[mask]

if 'price(in lakhs)' in df.columns:
    df = remove_outliers(df, 'price(in lakhs)')


In [32]:
df.shape

(1362, 4)

In [33]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [36]:
num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns


num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )


cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder(handle_unknown='ignore'))])

In [37]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

#Завдання 3

In [59]:
from sklearn.tree import DecisionTreeRegressor
model = Pipeline(
    [("prep", preprocessor),
     ("model", DecisionTreeRegressor(max_depth=2,
                                      random_state=42)
     )
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=2, random_state=42))])

In [60]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=2, random_state=42))])

# Завдання 4
Виведіть основні метрики моделі

In [61]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [62]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, model.predict(X_test))

all_metrics

,train,test
MAE,3.66,3.90
RMSE,5.29,5.73
R2,0.45,0.36


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

In [63]:
all_metrics = pd.DataFrame()

for depth in range(3, 19):
    model = Pipeline([
        ("prep", preprocessor),
        ("model", DecisionTreeRegressor(max_depth=depth, random_state=42))
    ])

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_metrics = get_metrics(y_train, y_train_pred, name=f"train_depth_{depth}")
    test_metrics = get_metrics(y_test, y_test_pred, name=f"test_depth_{depth}")

    all_metrics = pd.concat([all_metrics, train_metrics, test_metrics], axis=1)

all_metrics.round(2)

,train_depth_3,test_depth_3,train_depth_4,test_depth_4,train_depth_5,test_depth_5,train_depth_6,test_depth_6,train_depth_7,test_depth_7,...,train_depth_14,test_depth_14,train_depth_15,test_depth_15,train_depth_16,test_depth_16,train_depth_17,test_depth_17,train_depth_18,test_depth_18
MAE,3.01,3.59,2.70,3.04,2.34,2.80,2.18,2.58,1.89,2.50,...,0.45,1.68,0.36,1.62,0.28,1.66,0.21,1.67,0.19,1.70
RMSE,4.71,5.65,4.12,4.72,3.69,4.51,3.44,4.14,3.10,4.14,...,1.03,3.12,0.83,3.05,0.72,3.40,0.62,3.42,0.59,3.49
R2,0.57,0.38,0.67,0.57,0.73,0.61,0.77,0.67,0.81,0.67,...,0.98,0.81,0.99,0.82,0.99,0.78,0.99,0.77,0.99,0.76


# Завдання 6
Збережіть модель

In [64]:
import joblib

joblib.dump(model, 'new_model.joblib')

new_model = joblib.load('new_model.joblib')

new_model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=18, random_state=42))])